<a href="https://colab.research.google.com/github/Dkghuser/Movie-Recommendation-System/blob/main/Movie_Recommendation_System_using_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Movie Recommendation System**

Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data collection and data Pre-Processing

In [2]:
# loading the data from csv file to pandas dataframe
moviedata = pd.read_csv('/content/movies.csv')

In [3]:
# printing the first 5 rows of the dataframe
moviedata.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# to find the no. of rows & columns in the dataframe
moviedata.shape

(4803, 24)

In [12]:
# feature selection/selection of required features
features_sel = ['genres','keywords','tagline','release_date','overview']
print(features_sel)

['genres', 'keywords', 'tagline', 'release_date', 'overview']


In [13]:
#Replace null values with null string
for feature in features_sel:
  moviedata[feature] = moviedata[feature].fillna('')

In [14]:
#Combining the data of features selected
combined_features = moviedata['genres']+' '+moviedata['keywords']+' '+moviedata['overview']+' '+moviedata['release_date']+' '+moviedata['tagline']

In [15]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [16]:
# Convertion of text data to feature vectors
vectorizer = TfidfVectorizer()

In [18]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [19]:
print(feature_vectors)

  (0, 14181)	0.03617931695128806
  (0, 22387)	0.08256348455663665
  (0, 6908)	0.16467371630797395
  (0, 13)	0.08483403031730359
  (0, 26)	0.08570677629650786
  (0, 217)	0.1120864932831819
  (0, 3857)	0.18739069159720656
  (0, 859)	0.13947244029611425
  (0, 1017)	0.059834608289260904
  (0, 15882)	0.18901680776480792
  (0, 1047)	0.03627272098978272
  (0, 14364)	0.1830052145366333
  (0, 7975)	0.15720963983016853
  (0, 2256)	0.11322918856547241
  (0, 20509)	0.17192521928491578
  (0, 2066)	0.11131306695250981
  (0, 3058)	0.07033580570668499
  (0, 13180)	0.12418729768481088
  (0, 21205)	0.17585549802904443
  (0, 14265)	0.05572068885205959
  (0, 14677)	0.4285199035149865
  (0, 13338)	0.17585549802904443
  (0, 20424)	0.03520914385564598
  (0, 5946)	0.2041128036133696
  (0, 10715)	0.04734009619867613
  :	:
  (4802, 206)	0.0869163547858348
  (4802, 9074)	0.10035276233022478
  (4802, 22060)	0.0991525560320845
  (4802, 7129)	0.09106163578050847
  (4802, 14231)	0.07385433842087202
  (4802, 22512)	0

Cosine similarity

In [20]:
# Getting similarity scores by using cosine similarity, Similar movies

Similarity = cosine_similarity(feature_vectors)

In [21]:
print(Similarity)

[[1.         0.06880098 0.03539731 ... 0.03950119 0.01585561 0.01055399]
 [0.06880098 1.         0.05956098 ... 0.06664116 0.04742301 0.02964248]
 [0.03539731 0.05956098 1.         ... 0.03819205 0.0148713  0.02017459]
 ...
 [0.03950119 0.06664116 0.03819205 ... 1.         0.04254107 0.03999921]
 [0.01585561 0.04742301 0.0148713  ... 0.04254107 1.         0.04865178]
 [0.01055399 0.02964248 0.02017459 ... 0.03999921 0.04865178 1.        ]]


In [22]:
print(Similarity.shape)

(4803, 4803)


Getting the Movie name as i/p from user

In [32]:
# getting/Enter movie name from the user

Movie_name = input('Enter your favourite movie name : ')

Enter your favourite movie name : iron man


Create a movie list from dataset

In [24]:
# Create a list of all the movie names present in the dataset

list_of_all_movititles = moviedata['title'].tolist()
print(list_of_all_movititles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [25]:
# Finding the best/close match for the movie name entered by the user
find_best_match = difflib.get_close_matches(Movie_name, list_of_all_movititles)
print(find_best_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [26]:
# To get only 1 value from the list
best_match = find_best_match[0]
print(best_match)

Iron Man


In [27]:
# Finding the index of the movie with title
index_of_the_movie = moviedata[moviedata.title == best_match]['index'].values[0]
print(index_of_the_movie)

68


In [29]:
# getting a list of the similar movies
Similarity_score = list(enumerate(Similarity[index_of_the_movie]))
print(Similarity_score)

[(0, 0.06525031503803193), (1, 0.04985578341031426), (2, 0.014486352968882146), (3, 0.007897857024106798), (4, 0.03270626838938813), (5, 0.01799296023228926), (6, 0.02683555106908931), (7, 0.21619418258309664), (8, 0.013596933542157292), (9, 0.04602484342801416), (10, 0.04855927292134182), (11, 0.037816506258524285), (12, 0.014049245280867075), (13, 0.0130248118451416), (14, 0.07460288008250192), (15, 0.04392304577681935), (16, 0.12607041116216514), (17, 0.019374508273860697), (18, 0.025742849682954445), (19, 0.050917261381982264), (20, 0.04235410095393209), (21, 0.013252513380767305), (22, 0.011616545110173829), (23, 0.029815725738012017), (24, 0.022699090537641374), (25, 0.004061799981131525), (26, 0.10542589565196783), (27, 0.04571719919312263), (28, 0.029256212023543796), (29, 0.011558591124537151), (30, 0.09643369842308391), (31, 0.27586751066579834), (32, 0.03086803868116259), (33, 0.10181960180662869), (34, 0.007815605797551714), (35, 0.027880158802365426), (36, 0.03150888453655

In [30]:
len(Similarity_score)

4803

In [31]:
# sorting of the movies based on their Similarity score, (desc.order)

sorted_simlr_movies = sorted(Similarity_score, key = lambda x:x[1], reverse=True)
print(sorted_simlr_movies)

[(68, 1.0), (79, 0.29168917301352165), (31, 0.27586751066579834), (7, 0.21619418258309664), (511, 0.15470928223049243), (64, 0.1473066262966213), (94, 0.14718011720853869), (182, 0.13123925529602207), (16, 0.12607041116216514), (174, 0.11887144991627), (4033, 0.11422893217862375), (46, 0.11235432991015194), (618, 0.10794621480652936), (122, 0.10737348886423619), (420, 0.1065252113026888), (26, 0.10542589565196783), (33, 0.10181960180662869), (203, 0.10030488892711265), (76, 0.09953706244032147), (101, 0.09708644197080607), (30, 0.09643369842308391), (4399, 0.09619349613618211), (3606, 0.0952830563255258), (1654, 0.09479761903331631), (2044, 0.09402518017454713), (1135, 0.09315816496337474), (1177, 0.09288555814399437), (1868, 0.09280515268468309), (782, 0.09273042484428866), (85, 0.09170119229372467), (788, 0.09145631478108227), (3133, 0.08895237118935882), (1569, 0.08851550561453592), (1740, 0.08806965823880583), (1628, 0.08799569571716019), (394, 0.08741129186891097), (2229, 0.087354

In [45]:
# Display/print the name of the similar movies based on the index value

print('Recommended Movies for you : \n')

i=1

for movie in sorted_simlr_movies:
  index=movie[0]
  title_from_index = moviedata[moviedata.index == index]['title'].values[0]
  if(i<21):
    print(i,'.', title_from_index)
    i+=1

Recommended Movies for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . X-Men
6 . X-Men: Apocalypse
7 . Guardians of the Galaxy
8 . Ant-Man
9 . The Avengers
10 . The Incredible Hulk
11 . Super
12 . X-Men: Days of Future Past
13 . Mystery Men
14 . X-Men Origins: Wolverine
15 . Hellboy II: The Golden Army
16 . Captain America: Civil War
17 . X-Men: The Last Stand
18 . X2
19 . G.I. Joe: The Rise of Cobra
20 . X-Men: First Class


Movie Recommendation System

In [46]:
Movie_name = input('Enter your favourite movie name : ')
list_of_all_movititles = moviedata['title'].tolist()
find_best_match = difflib.get_close_matches(Movie_name, list_of_all_movititles)
best_match = find_best_match[0]
index_of_the_movie = moviedata[moviedata.title == best_match]['index'].values[0]
Similarity_score = list(enumerate(Similarity[index_of_the_movie]))
sorted_simlr_movies = sorted(Similarity_score, key = lambda x:x[1], reverse=True)
print('Recommended Movies for you : \n')

i=1

for movie in sorted_simlr_movies:
  index=movie[0]
  title_from_index = moviedata[moviedata.index == index]['title'].values[0]
  if(i<21):
    print(i,'.', title_from_index)
    i+=1

Enter your favourite movie name : batman
Recommended Movies for you : 

1 . Batman
2 . Batman Begins
3 . Batman & Robin
4 . The Dark Knight Rises
5 . Batman Returns
6 . The Dark Knight
7 . Batman: The Dark Knight Returns, Part 2
8 . Batman Forever
9 . Batman v Superman: Dawn of Justice
10 . A History of Violence
11 . Spider-Man 2
12 . Suicide Squad
13 . Chill Factor
14 . Superman
15 . Salton Sea
16 . Green Lantern
17 . Creepshow
18 . Superman II
19 . Red Tails
20 . Bedazzled
